In [1]:
import pandas as pd
import numpy as np

In [2]:
import re

In [3]:
df=pd.read_csv('/content/chat_dataset.csv',encoding='UTF-8')
df.head()

,message,sentiment
0,I really enjoyed the movie,positive
1,The food was terrible,negative
2,I'm not sure how I feel about this,neutral
3,The service was excellent,positive
4,I had a bad experience,negative


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   message    584 non-null    object
 1   sentiment  584 non-null    object
dtypes: object(2)
memory usage: 9.2+ KB


In [5]:
df['sentiment'].value_counts()

neutral     259
positive    178
negative    147
Name: sentiment, dtype: int64

In [6]:
def ohe(text):

    if text=='positive':
         return 10
    elif text=='neutral':
         return 5
    elif text=='negative' :
          return 1
    return 1


In [7]:
df['sentiment']=df['sentiment'].apply(ohe)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   message    584 non-null    object
 1   sentiment  584 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.2+ KB


**Remove links**

In [9]:
def remove_link(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    if pattern:
        return pattern.sub(r'', text)
    else:
        return text



In [10]:
import string
Exclude=string.punctuation
Exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

**Remove Punctuation**

In [11]:
def remove_punctuation(text):
  return text.translate(str.maketrans('','',Exclude))

**Remove HTML Tag**

In [12]:
def remove_html_tag(text):
  i=0
  pattern=re.compile(r'<.*?>')
  if pattern:
    list=np.array(pattern.findall(text))
    n=list.shape[0]
    for i in range(n):
      text= text.replace(list[i],'')
    return text


**Spelling Correction**

In [13]:
from textblob import TextBlob
def correct_spell(text):
  textblb=TextBlob(text)
  return textblb.correct().string

In [14]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
extra=stopwords.words('english')
stopwrd=np.array(extra)
stopwrd.shape

(179,)

**Removing stopwards**

In [16]:
def remove_stopwords(text, stopwords):
    split_text = text.split()
    filtered_text = []

    for word in split_text:
        if word not in stopwords:
            filtered_text.append(word)

    result = ' '.join(filtered_text)
    return result


**Remove emojis**

In [17]:
def remove_emoji(text):
    pattern=re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    if pattern:
       list= pattern.findall(text)
    return pattern.sub(r'', text)


In [18]:
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatization(text):
    doc=nlp(text)
    text_=[]
    for word in doc:
       text_.append(word.lemma_)

    result = ' '.join(text_)
    return result




In [19]:
def text_cleaning(text,stopwords):
    text1=remove_link(text)
    text2=remove_html_tag(text1)
    text3=remove_emoji(text2)
    text4=remove_stopwords(text3,stopwords)
    text5=remove_punctuation(text4)
    text6=lemmatization(text5)
    text7=correct_spell(text6)
    return text7

In [20]:
df['message']=df['message'].apply(lambda x:text_cleaning(x,extra))

In [21]:
df

,message,sentiment
0,I really enjoy movie,10
1,the food terrible,1
2,I m sure I feel,5
3,the service excellent,10
4,I bad experience,1
...,...,...
579,I cancel vacation plan I can not get time work,1
580,my computer crash I lose important file,1
581,I get car accident car total,1
582,I cold can not stop cough really hard sleep,1


In [22]:
tweet=[]
def formlist(text,tweet):
    tweet.append(text)

In [23]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
df['message'].apply(lambda x:formlist(x,tweet))

In [25]:
def total_words(list):
    tokenizer=Tokenizer(num_words=100)
    tokenizer.fit_on_texts(list)
    return max(tokenizer.word_index.values())
total_tweet_word=total_words(tweet)
tweet_tokenizer=Tokenizer(total_tweet_word)
tweet_tokenizer.fit_on_texts(tweet)

In [26]:
def tokenize_pad(list,max_len,num_words):
    tokenizer=Tokenizer(num_words=num_words)
    tokenizer.fit_on_texts(list)
    sequence=tokenizer.texts_to_sequences(list)
    padded=pad_sequences(sequence,padding='post',maxlen=max_len)
    return padded

In [27]:
tweet_pad=tokenize_pad(tweet,max(df['message'].str.split().str.len()),total_tweet_word)
tweet_pad

array([[  1,  29,  25, ...,   0,   0,   0],
       [  3,  17,  36, ...,   0,   0,   0],
       [  1,   2,   7, ...,   0,   0,   0],
       ...,
       [  1,  16,  37, ...,   0,   0,   0],
       [  1, 583,  47, ...,   0,   0,   0],
       [  1, 112, 157, ...,   0,   0,   0]], dtype=int32)

#Model Architecture

In [28]:
from keras.layers import Input, Embedding, BatchNormalization, LSTM, Dense
from keras.models import Model
from keras.optimizers import Nadam
input=Input(shape=(14,),name='Tweets_Input')
x = Embedding(total_tweet_word, 100,name='Embedding_layer')(input)
x = BatchNormalization(name='Normalization_layer')(x)
x=LSTM(100,return_sequences=True,name='Encoder_LSTM1')(x)

x=Dense(100,activation='relu',name='Dense_layer_1')(x)
x=Dense(50,activation='relu',name='Dense_layer_2')(x)
output=Dense(1,activation='relu',name='output_layer')(x)
model=Model(inputs=input,outputs=output)
loss_fn=keras.losses.MAE

model.compile(optimizer=Nadam(learning_rate=0.001), loss=loss_fn)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Tweets_Input (InputLayer)   [(None, 14)]              0         
                                                                 
 Embedding_layer (Embedding  (None, 14, 100)           58700     
 )                                                               
                                                                 
 Normalization_layer (Batch  (None, 14, 100)           400       
 Normalization)                                                  
                                                                 
 Encoder_LSTM1 (LSTM)        (None, 14, 100)           80400     
                                                                 
 Dense_layer_1 (Dense)       (None, 14, 100)           10100     
                                                                 
 Dense_layer_2 (Dense)       (None, 14, 50)            5050  

In [ ]:
batch_size=100
epochs=100
history=model.fit(tweet_pad,df['sentiment'] ,batch_size=batch_size,epochs=epochs,validation_split=0.12)

In [35]:
def input_tokenize_pad(input_text,tokenizer,max_len):
    sequence=tokenizer.texts_to_sequences([input_text])
    padded=pad_sequences(sequence,padding='post',maxlen=max_len)
    return padded
def predict(x):
    if (x>0 and x<3):
       return "negative sentiment"
    elif(x>3 and x<7):
      return "neutral sentiment"
    return "positive sentiment"

In [36]:
text=input_tokenize_pad("i am happy",tweet_tokenizer,14)
x=np.mean(max(model(text)).numpy())
predict(x)

'positive sentiment'

In [39]:
text=input_tokenize_pad("i am unhappy",tweet_tokenizer,14)
x=np.mean(max(model(text)).numpy())
predict(x)

'negative sentiment'

In [41]:
text=input_tokenize_pad("i am unhappy and happy",tweet_tokenizer,14)
x=np.mean(max(model(text)).numpy())
predict(x)

'neutral sentiment'